<center>

# ***MVP de Machine Learning e Analytics - Uma abordagem multitarefa para a Criptoanálise Clássica: Uma homenagem aos Pioneiros das Redes Neurais Artificiais, de McCullogh e Pitts à Alan Turing***

> ## ***Decifrando a Bíblia Sagrada cifrada por César e Vigenère***



</center>

### ***Nome:*** **Luís Gabriel Nascimento Simas**

### ***Matrícula:*** **4052025000943**

# 0. **Apresentação**

## 0.1. **Temos atração pelo desconhecido**
> ### A natureza humana sempre foi atraída pelo desconhecido. Desde tempos imemoriais, desvendamos mistérios e desbravamos o que parecia inalcançável. O mesmo princípio se aplica à ciência e à tecnologia. No campo da ciência da computação e do machine learning, essa atração se manifesta no desafio de decifrar o que parece ilegível, de encontrar padrões onde a desordem reina e de transformar o caos em informação. **Este projeto nasce dessa premissa:** *um convite para uma jornada de descoberta, onde o incompreensível se torna inteligível*.

## 0.2. **Por quê usar cifras?**
> ### As cifras, em sua essência, são a manifestação da nossa necessidade de proteger o que é valioso. Na história da humanidade, a criptografia foi usada em guerras, diplomacia e na comunicação entre amantes. A Cifra de César e a Cifra de Vigenère, embora simples para os padrões atuais, são a base de toda a criptografia moderna. Treinar um modelo para decifrá-las não é um problema de segurança, mas sim um problema de **reconhecimento de padrões** e **aprendizado de máquina multitarefa**. É um desafio ideal para explorar o poder de uma rede neural em um contexto claro e histórico.

## 0.3. **Por quê a Bíblia Sagrada?**
> ### A escolha da Bíblia como base de dados para este projeto é deliberada. Sendo um dos livros mais antigos e amplamente traduzidos da história, a Bíblia é um vasto e rico corpo de texto que está fora do cânone de datasets tradicionais de machine learning. Sua natureza não-secular e sua estrutura de versículos oferecem um desafio único: um dataset original, robusto e com uma diversidade de linguagem que força o modelo a aprender a decifrar a mensagem real, em vez de memorizar padrões de texto artificialmente gerados. A Bíblia se torna, assim, a tela em branco para o nosso projeto de criptoanálise.

# 1. **Definição do Problema**

## 1.1. **Objetivo**
### O objetivo deste projeto é desenvolver e treinar um modelo de aprendizado de máquina capaz de performar a **criptoanálise de cifras históricas**. O modelo deve resolver duas tarefas simultaneamente a partir de um texto cifrado: **classificar** o tipo de cifra utilizada (Cifra de César ou Cifra de Vigenère) e **decodificar** o texto para sua forma original. A solução proposta emprega o aprendizado profundo (Deep Learning) com a construção de um modelo multitarefa.

## 1.2. **Premissas e Hipóteses**
### A principal hipótese é que uma rede neural artificial (RNN/LSTM) é capaz de identificar padrões complexos de criptografia e inferir a lógica de decodificação de forma autônoma. O modelo aprenderá, implicitamente, a identificar a distribuição de frequência dos caracteres em cada cifra para realizar a classificação e a decifração. Assumimos que a natureza previsível da Cifra de César e a lógica de repetição da Cifra de Vigenère são padrões que podem ser capturados por uma rede neural.

## 1.3. **Restrições e Condições**
### Para garantir a originalidade e a qualidade do projeto, foi criado um dataset original do zero, não utilizando nenhuma base de dados previamente vista em sala de aula. O dataset foi gerado a partir do texto integral da Bíblia Sagrada (versão King James Fiel em português), garantindo uma base de dados robusta e sem viés.

## 1.4. **Descrição do Dataset**
### O dataset foi gerado em formato **Parquet** para eficiência de armazenamento e leitura, um formato ideal para dados tabulares. Ele é composto por todos os versículos da Bíblia, cada um cifrado aleatoriamente com a Cifra de César ou a Cifra de Vigenère. O dataset contém as seguintes colunas:

-  ### `texto_original`: O versículo da Bíblia sem modificações.
-  ### `texto_cifrado`: O mesmo versículo, porém cifrado.
-  ### `tipo_cifra`: Uma etiqueta (label) que indica o tipo de cifra utilizada (`cesar` ou `vigenere`).
-  ### `chave_usada`: A chave utilizada na cifragem (o `shift` para César ou a chave de Vigenère).

# 2. **Preparação dos dados**

## 2.1. **Objetivo**
### Gerar um dataset original e robusto para o treinamento e a avaliação do modelo.

## 2.2. **Carga e Preparação**
### Este projeto difere de abordagens que usam datasets prontos. Em vez disso, um dataset original foi criado do zero. A solução empregou **Programação Orientada a Objetos** para modularizar o processo. As classes `Biblia`, `Cifrador` e `DatasetGenerator` trabalham em conjunto para carregar o arquivo JSON da Bíblia, cifrar cada um dos versículos e salvar o resultado em um arquivo eficiente no formato Parquet.

### Baixando o repositório do github

In [ ]:
!git clone https://github.com/gabrielsimas/biblia-cifra-cesar-vigenere.git

fatal: destination path 'biblia-cifra-cesar-vigenere' already exists and is not an empty directory.


### **Instalando os pacotes para o projeto**

In [ ]:
import json
from dataclasses import dataclass, field
from typing import List, Optional
import random

#### **Funções e Classes auxiliares**
#### **Aqui estão todas as funções e classes utilizadas no Projeto**

In [ ]:
BIBLIA_JSON_PATH = '/content/biblia-cifra-cesar-vigenere/KJA.json'

In [ ]:
@dataclass
class Livro:
  abbrev: str
  chapters: List[List[str]]
  name: str

In [ ]:
from os import lchown
@dataclass
class Biblia:
  livros: List[Livro] = field(default_factory=list)

  def carregar_arquivo_biblia_json(self, arquivo_json: str):
    with open(arquivo_json, 'r', encoding='utf-8') as f:
      dados_json = json.load(f)
      self.livros = [Livro(**livro) for livro in dados_json]

  def escolher_versiculo(self, livro_abbrev: Optional[str] = None, capitulo_num: Optional[int] = None) -> str:
    """
    Seleciona e retorna um versículo aleatório. Pode ser filtrado por livro e capítulo.

    Args:
        livro_abbrev (Optional[str]): A abreviação do livro (ex: 'Gn'). Se None, será aleatório.
        capitulo_num (Optional[int]): O número do capítulo (ex: 1). Se None, será aleatório.

    Returns:
        str: Um versículo formatado como "Livro Capítulo:Versículo Texto".
    """
    livros_filtrados = self.livros

    # 1. Escolhe o livro
    if livro_abbrev:
        livros_encontrados = [l for l in self.livros if l.abbrev.lower() == livro_abbrev.lower()]
        if not livros_encontrados:
            raise ValueError(f"Livro com abreviação '{livro_abbrev}' não encontrado.")
        livro_escolhido = livros_encontrados[0]
    else:
        livro_escolhido = random.choice(self.livros)

    # 2. Escolhe o capítulo
    capitulos_do_livro = livro_escolhido.chapters
    if capitulo_num:
        if 0 < capitulo_num <= len(capitulos_do_livro):
            capitulo_escolhido = capitulos_do_livro[capitulo_num - 1]
        else:
            raise ValueError(f"Capítulo {capitulo_num} não encontrado no livro de {livro_escolhido.abbrev}.")
    else:
        capitulo_escolhido = random.choice(capitulos_do_livro)
        capitulo_num = livro_escolhido.chapters.index(capitulo_escolhido) + 1

    # 3. Escolhe o versículo
    versiculo_escolhido = random.choice(capitulo_escolhido)
    numero_do_versiculo = capitulo_escolhido.index(versiculo_escolhido) + 1

    # Formata o output como "nome do livro numero do capitulo: número do versículo: texto do versículo"
    return f"{livro_escolhido.name} {capitulo_num}:{numero_do_versiculo}: {versiculo_escolhido}"


In [ ]:
class Cifrador():
  def __init__(self, texto: str) -> None:
    self._texto_original = texto
    self._texto_atual = texto
    self._esta_cifrada = False

  @property
  def texto_atual(self) -> str:
    """Getter que retorna o texto no seu estado atual."""
    return self._texto_atual

  @texto_atual.setter
  def texto_atual(self, novo_texto: str):
    """Setter que atualiza o texto atual."""
    self._texto_atual = novo_texto

  @property
  def texto_original(self) -> str:
    """Getter que retorna o texto original."""
    return self._texto_original

  def converte_minuscula(self):
    """
    Converte o texto atual para minúsculas, mas apenas se ele não estiver cifrado.
    """
    if not self._esta_cifrada:
      self.texto_atual = self.texto_atual.lower()
    else:
      print("Erro: Não é possível converter para minúsculas. O texto já está cifrado.")

  def encode_cesar(self, shift: int) -> str:
    """
    Codifica o texto atual usando a Cifra de César, preservando a capitalização.
    """
    resultado = ""
    for char in self._texto_atual:
      if 'a' <= char <= 'z':
        nova_posicao = (ord(char) - ord('a') + shift) % 26
        resultado += chr(ord('a') + nova_posicao)
      elif 'A' <= char <= 'Z':
        nova_posicao = (ord(char) - ord('A') + shift) % 26
        resultado += chr(ord('A') + nova_posicao)
      else:
        resultado += char

    self.texto_atual = resultado
    self._esta_cifrada = True
    return self.texto_atual

  def encode_vigenere(self, chave: str) -> str:
    """
    Codifica o texto atual usando a Cifra de Vigenère, preservando a capitalização.
    """
    resultado = ""
    chave = chave.lower()
    indice_chave = 0

    for char in self._texto_atual:
      if 'a' <= char <= 'z':
        shift_vigenere = ord(chave[indice_chave]) - ord('a')
        nova_posicao = (ord(char) - ord('a') + shift_vigenere) % 26
        resultado += chr(ord('a') + nova_posicao)
        indice_chave = (indice_chave + 1) % len(chave)
      elif 'A' <= char <= 'Z':
        shift_vigenere = ord(chave[indice_chave]) - ord('a')
        nova_posicao = (ord(char) - ord('A') + shift_vigenere) % 26
        resultado += chr(ord('A') + nova_posicao)
        indice_chave = (indice_chave + 1) % len(chave)
      else:
        resultado += char

    self.texto_atual = resultado
    self._esta_cifrada = True
    return self._texto_atual

  def decode_cesar(self, shift: int) -> str:
    """
    Decodifica o texto atual usando a Cifra de César.
    """
    self._esta_cifrada = False
    return self.encode_cesar(-shift)

  def decode_vigenere(self, chave: str) -> str:
    """
    Decodifica o texto atual usando a Cifra de Vigenère.
    """
    resultado = ""
    chave = chave.lower()
    indice_chave = 0

    for char in self._texto_atual:
      if 'a' <= char <= 'z':
        shift_vigenere = ord(chave[indice_chave]) - ord('a')
        nova_posicao = (ord(char) - ord('a') - shift_vigenere) % 26
        resultado += chr(ord('a') + nova_posicao)
        indice_chave = (indice_chave + 1) % len(chave)
      elif 'A' <= char <= 'Z':
        shift_vigenere = ord(chave[indice_chave]) - ord('a')
        nova_posicao = (ord(char) - ord('A') - shift_vigenere) % 26
        resultado += chr(ord('A') + nova_posicao)
        indice_chave = (indice_chave + 1) % len(chave)
      else:
        resultado += char

    self._esta_cifrada = False
    self.texto_atual = resultado
    return self.texto_atual

  def reset(self) -> str:
    """
    Reseta o texto atual para o texto original.
    """
    self._esta_cifrada = False
    self.texto_atual = self.texto_original
    return self.texto_atual

In [ ]:
import unicodedata
import re

def extrair_chave_da_citacao(texto_completo: str) -> str:
  """
    Extrai a citação de um versículo, remove a acentuação e outros caracteres, e retorna apenas as letras.

    Args:
        texto_completo (str): O versículo completo, incluindo a citação (ex: "3 João 1:3: ...").

    Returns:
        str: Apenas as letras da citação, em minúsculas e sem acentuação (ex: "joao").
  """
  match = re.search(r'^(.*?):', texto_completo)

  if match:
    citacao_bruta = match.group(1)
    citacao_sem_acento = unicodedata.normalize('NFKD', citacao_bruta).encode('ascii','ignore').decode('utf-8')
    chave_limpa = re.sub(r'[^a-zA-Z]','', citacao_sem_acento)
    return chave_limpa.lower()

  return ""

In [ ]:
@dataclass
class AmostraDataset:
  """
  Representa uma única amostra de dados para o dataset.
  """
  texto_original: str
  texto_cifrado: str
  tipo_cifra: str
  chave_usada: Optional[str] = None

In [ ]:
import pandas as pd

class GeradorConjuntoDados:
  def __init__(self, biblia: Biblia, cifrador: Cifrador) -> None:
    self._biblia = biblia
    self._cifrador = cifrador

  def _extrair_e_limpar_citacao(self, versiculo_completo: str) -> tuple[str, str]:
    """
    Extrai a citação de um versículo, remove a acentuação e outros caracteres, e retorna apenas as letras.

    Args:
        texto_completo (str): O versículo completo, incluindo a citação (ex: "3 João 1:3: ...").

    Returns:
        str: Apenas as letras da citação, em minúsculas e sem acentuação (ex: "joao").
    """
    partes = versiculo_completo.split(' ', 2)
    if len(partes) < 3:
      return "", ""

    citacao_completa = " ".join(partes[:2])
    texto_original = partes[2]

    citacao_sem_acento = unicodedata.normalize('NFKD', citacao_completa).encode('ascii','ignore').decode('utf-8')
    citacao_limpa = re.sub(r'[^a-zA-Z]', '', citacao_sem_acento)

    chave_final = citacao_limpa.lower()

    return chave_final, texto_original

  def _processar_cesar(self, texto_original: str) -> dict:
    self._cifrador.texto_atual = texto_original
    shift = random.randint(1, 25)
    texto_cifrado = self._cifrador.encode_cesar(shift)
    return AmostraDataset(
        texto_original=texto_original,
        texto_cifrado=texto_cifrado,
        tipo_cifra="cesar",
        chave_usada=str(shift)
    )

  def _processar_vigenere(self, texto_original: str, chave_base: str) -> dict:
    self._cifrador.texto_atual = texto_original
    shift_cesar_chave = random.randint(1, 25)

    chave_final = self._cifrador.encode_cesar(shift=shift_cesar_chave)
    texto_cifrado = self._cifrador.encode_vigenere(chave=chave_final)

    return AmostraDataset(
        texto_original=texto_original,
        texto_cifrado=texto_cifrado,
        tipo_cifra="cesar",
        chave_usada=chave_final
    )

  def gerar_conjunto_dados(self, nome_arquivo: str = "dataset_biblia_criptografada.parquet"):
    """
      Gera um dataset completo com todos os versículos da Bíblia,
      codificados com as cifras de César ou Vigenère, de forma aleatória,
      e salva o resultado em um arquivo Parquet.

      Args:
          nome_arquivo (str): O nome do arquivo Parquet a ser salvo.
    """
    amostras = []

    todos_os_versiculos = (versiculo for livro in self._biblia.livros
                            for capitulo in livro.chapters for versiculo in capitulo)

    for versiculo_completo in todos_os_versiculos:
      citacao_limpa, texto_original = self._extrair_e_limpar_citacao(versiculo_completo)

      if not text_original:
        continue

      cifra_escolhida = random.choice(["cesar", "vigenere"])

      if cifra_escolhida == "cesar":
        amostra = self._processar_cesar(texto_original)
      else:
        amostra = self._processar_vigenere(texto_original, citacao_limpa)

      amostras.append(amostra)

    df_dataset = pd.DataFrame(amostras)
    df_dataset.to_parquet(nome_arquivo, index=False)
    print(f"Dataset gerado e salvo com sucesso em '{nome_arquivo}'!")


#### **Criação do dataset**

In [ ]:
biblia_obj = Biblia()
cifrador_obj = Cifrador("")

In [ ]:
biblia_obj.carregar_arquivo_biblia_json(BIBLIA_JSON_PATH)

In [ ]:
gerador = GeradorConjuntoDados(biblia=biblia_obj, cifrador=cifrador_obj)
gerador.gerar_conjunto_dados(nome_arquivo='/content/biblia-cifra-cesar-vigenere/dataset_biblia_criptografada.parquet')

Dataset gerado e salvo com sucesso em '/content/biblia-cifra-cesar-vigenere/dataset_biblia_criptografada.parquet'!


## 2.3. **Divisão dos Dados**
### Após a geração, o dataset será dividido em três partes: treino, validação e teste, em proporções adequadas para garantir que o modelo seja treinado, ajustado e avaliado em dados não vistos. Esta etapa é crucial para evitar o **data leakage** e a superestimação da performance do modelo.

## 2.4. **Tratamento de Dados**
### Como o projeto gera dados limpos, o foco do tratamento de dados será na transformação das sequências de texto em representações numéricas (vetores), a linguagem que a rede neural entende. Será utilizada a técnica de **one-hot encoding** para criar uma representação numérica de cada caractere.

# 3. **Modelagem e Treinamento**

## 3.1. **Objetivo**
### Construir e treinar um modelo de aprendizado profundo capaz de realizar duas tarefas simultâneas: classificar o tipo de cifra e decifrar o texto.

## 3.2. **Seleção dos Algoritmos**
### A solução empregará uma arquitetura de rede neural do tipo **Encoder-Decoder**. O Encoder será uma Rede Neural Recorrente (RNN) ou uma Long Short-Term Memory (LSTM), ideal para processar sequências de texto. O modelo terá duas "cabeças" de saída, uma para cada tarefa: uma cabeça de classificação (utilizando uma camada de saída com ativação `softmax`) e uma cabeça de decodificação (`dense` + `softmax`).

## 3.3 **Otimização de Hiperparâmetros**
### Serão explorados hiperparâmetros como a taxa de aprendizado (`learning rate`), o número de camadas ocultas e o tamanho dos vetores de entrada, com o objetivo de otimizar a performance do modelo.

## 3.4. **Treinamento**
### O modelo será treinado com o dataset gerado, utilizando as saídas para a classificação da cifra e a decodificação do texto. Serão fixadas as *seeds* para garantir a reprodutibilidade dos resultados.

# **4. Avaliação de Resultados**

## 4.1. **Objetivo**
### Analisar o desempenho dos modelos gerados em dados não vistos (com a base de teste).

## 4.2. **Métricas de Avaliação:**
-   ### **Classificação (Tipo de Cifra):** Será utilizada a **Acurácia**, medindo a porcentagem de textos que o modelo classifica corretamente.
-   ### **Decodificação (Texto Decifrado):** A performance será avaliada usando a **Acurácia de Caracteres** (quantos caracteres foram decifrados corretamente na posição certa) e o **BLEU Score** (uma métrica que compara a similaridade da frase decifrada com a frase original).

## 4.3 **Análise e Comparação:**
### Os resultados dos modelos serão analisados e comparados para identificar o modelo com melhor desempenho. Será investigado o **underfitting** e **overfitting** para garantir que o modelo generaliza bem.

# 5. **Boas Práticas e Conclusão**

## 5.1. **Boas Práticas**
### O projeto segue as boas práticas de desenvolvimento, como **Programação Orientada a Objetos** (com classes separadas para `Biblia`, `Cifrador` e `DatasetGenerator`), **documentação consistente** e fixação de *seeds* para reprodutibilidade. As decisões de projeto estão documentadas textualmente em cada célula, contando a história do desenvolvimento.

## 5.2. **Conclusão**
### Este MVP é uma demonstração do poder do Deep Learning para resolver problemas complexos e multitarefa. O projeto é uma homenagem ao legado de Alan Turing e aos pioneiros das redes neurais, mostrando como o trabalho deles continua relevante e inspirador para a computação moderna.